# Research Questions: Open-Ended Explorations

**Track**: Extensions | **Difficulty**: ⭐⭐⭐⭐⭐ Research | **Time**: Variable (2-10 hours)

**Prerequisites**:
- Completed all Core Track notebooks
- Completed Extensions: Market Maker Simulation
- Completed Extensions: Liquidity Microstructure
- Strong quantitative background
- Independent research skills

**Learning Objectives**:
1. Formulate and test original hypotheses about market structure
2. Design quantitative experiments to answer open questions
3. Evaluate trade-offs in regulatory interventions
4. Extend existing models with novel features
5. Develop critical thinking about market design

**Key Concepts**:
- Counterfactual analysis
- Market design trade-offs
- Regulatory policy evaluation
- Model extensions and validation

---

## Important Note

**These questions have NO single "correct" answer.**

The goal is to:
- Think critically about market structure
- Make reasonable assumptions and justify them
- Build models and test hypotheses
- Evaluate trade-offs and second-order effects
- Present findings clearly

Your analysis methods matter more than your conclusions.

---

## Setup

In [1]:
# ============================================================================
# SETUP - Research Questions
# ============================================================================

import sys
import os
from pathlib import Path

# Detect environment
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("💻 Running in local Jupyter environment")

# Setup for Colab
if IN_COLAB:
    print("\n📥 Setting up Colab environment...")
    
    if not os.path.exists('/content/etf-flash-crash-2015'):
        print("  Cloning repository...")
        !git clone https://github.com/shiong-tan/etf-flash-crash-2015.git /content/etf-flash-crash-2015
        print("  ✅ Repository cloned")
    else:
        print("  ✅ Repository exists")
    
    print("  Installing dependencies...")
    !pip install -q pandas numpy matplotlib seaborn scipy
    print("  ✅ Dependencies installed")
    
    os.chdir('/content/etf-flash-crash-2015')
    sys.path.insert(0, '/content/etf-flash-crash-2015/src')
    
else:
    print("\n📂 Setting up local environment...")
    notebook_dir = Path().resolve()
    
    # Determine src directory location based on notebook location
    # Case 1: In notebooks/core/ or notebooks/extensions/ → go up 2 levels
    if notebook_dir.parent.name == 'notebooks':
        project_root = notebook_dir.parent.parent
        src_dir = project_root / 'src'
    # Case 2: In notebooks/ directory → go up 1 level
    elif notebook_dir.name == 'notebooks':
        project_root = notebook_dir.parent
        src_dir = project_root / 'src'
    # Case 3: In project root (has notebooks/ subdirectory) → use directly
    elif (notebook_dir / 'notebooks').exists():
        project_root = notebook_dir
        src_dir = project_root / 'src'
    # Case 4: Fallback - assume we're one level deep
    else:
        project_root = notebook_dir.parent
        src_dir = project_root / 'src'
    
    sys.path.insert(0, str(src_dir))
    print(f"  ✅ Added src to path: {src_dir}")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, optimize

from order_book_dynamics import (
    OrderBookSnapshot,
    FlashCrashOrderBook,
    calculate_kyle_lambda,
    calculate_amihud_illiquidity
)
from market_maker_pnl import MarketMakerSimulator, HedgeStatus

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)
np.random.seed(42)

if IN_COLAB:
    from IPython.display import set_matplotlib_formats
    set_matplotlib_formats('retina')
else:
    try:
        get_ipython().run_line_magic('matplotlib', 'inline')
    except:
        pass

print("\n" + "="*60)
print("✅ SETUP COMPLETE - Ready for research explorations!")
print("="*60)

💻 Running in local Jupyter environment

📂 Setting up local environment...
  ✅ Added src to path: /Users/shiongtan/projects/etf-flash-crash-2015/src



✅ SETUP COMPLETE - Ready for research explorations!


---

## Research Question 1: How Much Capital Could Have Prevented the Crash?

### Background

During the flash crash, market makers withdrew because they couldn't hedge inventory risk. If a well-capitalized stabilization fund had been available to absorb one-sided order flow, could the crash have been prevented?

### Your Task

Estimate the capital required to stabilize a single ETF (e.g., DVY) during the crash.

### Guiding Questions

- How much selling pressure occurred? (hint: volume during crash period)
- What inventory capacity would be needed?
- What is the VaR of holding that inventory?
- How long would positions need to be held?
- What if other ETFs also needed stabilization?
- Is this feasible? Who could provide such capital?

### Starter Code

In [2]:
# Research Question 1: Capital Requirements for Stabilization

# Example parameters (adjust based on your research)
etf_symbol = "DVY"
normal_price = 72.00
crash_low = 47.00
crash_duration_minutes = 20

# Estimate selling pressure
# TODO: Research actual volume data for August 24, 2015
normal_daily_volume = 500_000  # shares (placeholder)
crash_period_volume = None  # TODO: calculate
imbalance_ratio = None  # TODO: estimate buy/sell imbalance

# Calculate required inventory capacity
# TODO: How many shares would need to be absorbed?
required_inventory = None

# Calculate capital requirement
# TODO: inventory × price + VaR buffer
inventory_capital = None
var_buffer = None
total_capital = None

# Extend analysis:
# - What if applied to top 10 affected ETFs?
# - What about hedging costs?
# - What return would justify this capital deployment?

print("=== STABILIZATION CAPITAL ANALYSIS ===")
print(f"ETF: {etf_symbol}")
print(f"\nYour estimates:")
print(f"  Required inventory: {required_inventory:,} shares" if required_inventory else "  TODO: Calculate required inventory")
print(f"  Capital needed: ${total_capital:,}" if total_capital else "  TODO: Calculate capital")

# YOUR ANALYSIS HERE
# Write code to:
# 1. Estimate selling pressure
# 2. Calculate required inventory
# 3. Compute VaR for that position
# 4. Add hedging costs
# 5. Scale to multiple ETFs

=== STABILIZATION CAPITAL ANALYSIS ===
ETF: DVY

Your estimates:
  TODO: Calculate required inventory
  TODO: Calculate capital


### Discussion Prompts

After your analysis, consider:

1. **Feasibility**: Is the required capital realistic? Who could provide it?
2. **Moral hazard**: Would a stabilization fund encourage excessive risk-taking?
3. **Timing**: How quickly would the fund need to deploy capital?
4. **Profitability**: Could such a fund be profitable long-term?
5. **Alternatives**: Are there better solutions than direct capital injection?

---

## Research Question 2: Alternative Circuit Breaker Designs

### Background

LULD circuit breakers (5% bands) triggered repeatedly on August 24, but didn't prevent the crash. Could better circuit breaker design have helped?

### Your Task

Design and simulate alternative circuit breaker mechanisms.

### Possible Approaches

1. **Adaptive bands**: Widen bands when volatility is high
2. **Volume-based triggers**: Halt if volume surge exceeds threshold
3. **Liquidity-based triggers**: Halt when order book depth falls below threshold
4. **Coordinated halts**: Halt ETF when X% of components are halted
5. **Slow mode**: Allow trading but cap order rate

### Starter Code

In [3]:
# Research Question 2: Circuit Breaker Design

class CircuitBreaker:
    """Base class for circuit breaker mechanisms."""
    
    def __init__(self, name):
        self.name = name
        self.halt_count = 0
        self.halt_duration_total = 0
    
    def should_halt(self, **kwargs):
        """Override in subclass. Return True if trading should halt."""
        raise NotImplementedError

class LULDCircuitBreaker(CircuitBreaker):
    """Current LULD system (fixed percentage bands)."""
    
    def __init__(self, band_pct=5.0):
        super().__init__(f"LULD ({band_pct}%)")
        self.band_pct = band_pct
    
    def should_halt(self, price, reference_price, **kwargs):
        deviation_pct = abs((price - reference_price) / reference_price) * 100
        return deviation_pct >= self.band_pct

# TODO: Implement alternative circuit breakers

class AdaptiveCircuitBreaker(CircuitBreaker):
    """Adaptive bands that widen during high volatility."""
    
    def __init__(self, base_band_pct=5.0, max_band_pct=15.0):
        super().__init__("Adaptive LULD")
        self.base_band_pct = base_band_pct
        self.max_band_pct = max_band_pct
    
    def should_halt(self, price, reference_price, volatility=0.01, **kwargs):
        # TODO: Implement adaptive logic
        # Band widens as volatility increases
        # Example: band = base_band + (volatility * scaling_factor)
        pass

class LiquidityCircuitBreaker(CircuitBreaker):
    """Halt when order book depth falls below threshold."""
    
    def __init__(self, min_depth_shares=10000):
        super().__init__("Liquidity-Based")
        self.min_depth_shares = min_depth_shares
    
    def should_halt(self, order_book_depth=None, **kwargs):
        # TODO: Implement liquidity-based logic
        if order_book_depth is None:
            return False
        # Halt if depth falls below minimum
        pass

# TODO: Simulate flash crash with different circuit breakers
# Compare:
# - Number of halts triggered
# - Total halt duration
# - Maximum price deviation
# - Time to recovery
# - Liquidity during trading periods

print("=== CIRCUIT BREAKER COMPARISON ===")
print("\nTODO: Implement and compare different circuit breaker designs")
print("\nMetrics to track:")
print("  - Number of halts")
print("  - Total halt time")
print("  - Maximum price deviation")
print("  - Liquidity metrics (spread, depth)")
print("  - Time to price recovery")

=== CIRCUIT BREAKER COMPARISON ===

TODO: Implement and compare different circuit breaker designs

Metrics to track:
  - Number of halts
  - Total halt time
  - Maximum price deviation
  - Liquidity metrics (spread, depth)
  - Time to price recovery


### Discussion Prompts

1. **Trade-offs**: What are the costs and benefits of each design?
2. **Gaming**: Could traders game your circuit breaker mechanism?
3. **Coordination**: Should ETF halts be coordinated with underlying securities?
4. **Transparency**: Should circuit breaker parameters be public or secret?
5. **Unintended consequences**: What new problems might your design create?

---

## Research Question 3: Optimal Market Maker Obligations

### Background

Market makers withdrew during the crash because they had no obligation to maintain quotes during extreme conditions. Should they be required to?

### Your Task

Design a market maker obligation framework that balances:
- Continuous liquidity provision
- Protection from catastrophic losses
- Fair compensation for risk

### Starter Code

In [4]:
# Research Question 3: Market Maker Obligations

class MarketMakerObligation:
    """Framework for market maker obligations."""
    
    def __init__(self,
                 name,
                 min_quote_size,
                 max_spread_bps,
                 uptime_requirement_pct,
                 stress_exemption_rules):
        self.name = name
        self.min_quote_size = min_quote_size
        self.max_spread_bps = max_spread_bps
        self.uptime_requirement_pct = uptime_requirement_pct
        self.stress_exemption_rules = stress_exemption_rules
    
    def is_compliant(self, current_quotes):
        """Check if market maker is meeting obligations."""
        # TODO: Implement compliance checking
        pass
    
    def can_withdraw(self, market_conditions):
        """Check if market maker is exempt from obligations."""
        # TODO: Implement stress exemption logic
        pass

# Example frameworks to compare:

# Current system (no hard obligations)
current_system = MarketMakerObligation(
    name="Current (Voluntary)",
    min_quote_size=100,  # Nominal
    max_spread_bps=None,  # No limit
    uptime_requirement_pct=0,  # Can withdraw anytime
    stress_exemption_rules="Always exempt"
)

# Strict obligations (no exemptions)
strict_system = MarketMakerObligation(
    name="Strict (Always On)",
    min_quote_size=10000,
    max_spread_bps=10,  # 10 bps maximum
    uptime_requirement_pct=100,  # Must always quote
    stress_exemption_rules="No exemptions"
)

# TODO: Design your own framework
# Consider:
# - Normal vs stressed market requirements
# - VaR-based exemptions
# - Gradual withdrawal (widen spreads before full withdrawal)
# - Compensation for stress-period obligations

print("=== MARKET MAKER OBLIGATION FRAMEWORKS ===")
print("\nTODO: Design and evaluate obligation frameworks")
print("\nConsider:")
print("  1. How to define 'stress' conditions?")
print("  2. What spread widening should be allowed?")
print("  3. How to compensate for forced liquidity provision?")
print("  4. Penalties for non-compliance?")
print("  5. Would this attract or deter market makers?")

=== MARKET MAKER OBLIGATION FRAMEWORKS ===

TODO: Design and evaluate obligation frameworks

Consider:
  1. How to define 'stress' conditions?
  2. What spread widening should be allowed?
  3. How to compensate for forced liquidity provision?
  4. Penalties for non-compliance?
  5. Would this attract or deter market makers?


### Discussion Prompts

1. **Viability**: Would market makers accept your framework? What compensation is fair?
2. **Capital**: Would stricter obligations require more capital? Where would it come from?
3. **Competition**: How would obligations affect market maker competition?
4. **Systemic risk**: Could forced quoting during stress create catastrophic losses?
5. **International**: How do other markets handle this?

---

## Research Question 4: Stop-Loss Order Alternatives

### Background

Stop-loss orders amplified the crash. Many investors lost 30-40% when they intended to limit losses to 10%. What alternatives exist?

### Your Task

Compare stop-loss alternatives:
1. **Traditional stop-loss**: Becomes market order when trigger hit
2. **Stop-limit**: Becomes limit order (may not execute)
3. **Trailing stop**: Adjusts with favorable price movement
4. **Guaranteed stop**: Exchange guarantees execution price (for a fee)
5. **Dynamic protection**: Wider stop during high volatility

### Starter Code

In [5]:
# Research Question 4: Stop-Loss Alternatives

def simulate_stop_loss_outcome(price_path, entry_price, stop_type, **params):
    """
    Simulate outcome of different stop-loss strategies.
    
    Args:
        price_path: Array of prices over time
        entry_price: Original purchase price
        stop_type: 'traditional', 'limit', 'trailing', 'guaranteed', 'dynamic'
        **params: Stop-specific parameters
    
    Returns:
        dict with execution_price, execution_time, loss_pct
    """
    # TODO: Implement different stop-loss mechanisms
    pass

# Simulate August 24 price path for DVY
def generate_dvy_flash_crash_path():
    """Generate realistic price path for DVY on August 24."""
    # Opening: $72
    # Low: $47 (9:42 AM)
    # Recovery: $68 (10:30 AM)
    
    times = []
    prices = []
    
    # TODO: Create minute-by-minute price path
    # Model: gradual decline, sudden crash, partial recovery
    
    return pd.DataFrame({'time': times, 'price': prices})

# Compare outcomes
entry_price = 72.00
intended_stop = 0.10  # 10% loss

price_path = generate_dvy_flash_crash_path()

# TODO: Simulate each stop-loss type
# traditional_stop = simulate_stop_loss_outcome(price_path, entry_price, 'traditional', stop_pct=0.10)
# limit_stop = simulate_stop_loss_outcome(price_path, entry_price, 'limit', stop_pct=0.10, limit_pct=0.12)
# ...

print("=== STOP-LOSS ALTERNATIVES COMPARISON ===")
print(f"\nScenario: DVY purchased at ${entry_price:.2f}")
print(f"Investor intent: Limit loss to {intended_stop*100:.0f}%")
print(f"\nOutcomes on August 24, 2015:")
print("\nTODO: Implement and compare different stop-loss mechanisms")

=== STOP-LOSS ALTERNATIVES COMPARISON ===

Scenario: DVY purchased at $72.00
Investor intent: Limit loss to 10%

Outcomes on August 24, 2015:

TODO: Implement and compare different stop-loss mechanisms


### Discussion Prompts

1. **Education**: Do investors understand the difference between stop-loss types?
2. **Costs**: What should guaranteed stops cost? How to price them?
3. **Availability**: Should brokers offer more sophisticated stop types?
4. **Regulation**: Should traditional stops be banned during high volatility?
5. **Alternatives**: Are options a better hedging tool than stops?

---

## Research Question 5: ETF Structure Improvements

### Background

ETFs crashed because iNAV calculation failed when components halted. Could better ETF design prevent this?

### Your Task

Evaluate proposed ETF improvements:
1. **Stale price protection**: Don't trade if iNAV > X minutes old
2. **Reference NAV**: Use prior close NAV instead of stale iNAV
3. **Wider creation/redemption bands**: Allow more deviation before arbitrage
4. **Automatic halt**: Halt ETF when Y% of components halted
5. **Alternative weighting**: Equal-weight less susceptible to component halts?

### Starter Code

In [6]:
# Research Question 5: ETF Structure Improvements

def calculate_inav_reliability(components_trading_pct, time_since_last_update_seconds):
    """
    Estimate iNAV reliability based on component availability.
    
    Returns:
        reliability score 0-1 (1 = fully reliable)
    """
    # TODO: Model iNAV reliability degradation
    # Consider:
    # - Percentage of components with fresh prices
    # - Time since last update
    # - Volatility of halted components
    pass

def should_allow_etf_trading(inav_reliability, spread_to_inav_pct, trading_rule):
    """
    Decide whether ETF should trade given current conditions.
    
    Args:
        inav_reliability: 0-1 score
        spread_to_inav_pct: ETF price deviation from iNAV
        trading_rule: 'current', 'stale_protection', 'auto_halt', etc.
    
    Returns:
        bool: True if trading should continue
    """
    if trading_rule == 'current':
        # Current system: always trade
        return True
    
    elif trading_rule == 'stale_protection':
        # TODO: Implement stale price protection
        # Halt if iNAV reliability < threshold
        pass
    
    elif trading_rule == 'auto_halt':
        # TODO: Implement automatic halt
        # Halt if too many components halted
        pass
    
    return True

# TODO: Simulate August 24 with different trading rules
# Compare:
# - How much trading occurs?
# - Maximum price deviation
# - Investor losses prevented
# - Liquidity costs

print("=== ETF STRUCTURE IMPROVEMENTS ===")
print("\nTODO: Evaluate proposed ETF improvements")
print("\nQuestions to address:")
print("  1. How to measure iNAV reliability?")
print("  2. What reliability threshold for trading halt?")
print("  3. Trade-off: investor protection vs liquidity?")
print("  4. Would improvements prevent or worsen the crash?")
print("  5. Implementation: voluntary or regulatory mandate?")

=== ETF STRUCTURE IMPROVEMENTS ===

TODO: Evaluate proposed ETF improvements

Questions to address:
  1. How to measure iNAV reliability?
  2. What reliability threshold for trading halt?
  3. Trade-off: investor protection vs liquidity?
  4. Would improvements prevent or worsen the crash?
  5. Implementation: voluntary or regulatory mandate?


### Discussion Prompts

1. **Trade-offs**: Protection vs liquidity - where's the balance?
2. **Market impact**: Would automatic halts cascade across ETFs?
3. **Arbitrage**: Would wider bands reduce arbitrage efficiency?
4. **Competition**: Would complex rules favor large ETF issuers?
5. **Innovation**: Could new ETF structures eliminate these issues?

---

## Research Question 6: Your Own Question

### Design Your Own Research Project

Formulate and investigate an original question about the flash crash.

### Ideas

- **Contagion**: How did the crash spread across correlated ETFs?
- **Informed trading**: Were some traders better informed than others?
- **Optimal execution**: Best execution strategies during extreme volatility?
- **International comparison**: Why didn't similar crashes occur in other markets?
- **Long-term effects**: Did the flash crash permanently change market structure?
- **Prediction**: Can we predict flash crashes before they happen?

### Your Research

In [7]:
# Research Question 6: Your Original Research

# Define your research question
research_question = """
TODO: State your research question clearly.

Example: "Can network analysis of ETF correlations predict which ETFs 
will crash most severely during market stress?"
"""

# Formulate hypotheses
hypotheses = """
TODO: State testable hypotheses.

Example:
H1: ETFs with higher centrality in the correlation network will crash more.
H2: Less liquid ETFs will show stronger contagion effects.
"""

# Design your analysis
# TODO: Write code to test your hypotheses

print("=== YOUR RESEARCH PROJECT ===")
print(f"\nResearch Question:\n{research_question}")
print(f"\nHypotheses:\n{hypotheses}")
print("\nTODO: Implement your analysis")

=== YOUR RESEARCH PROJECT ===

Research Question:

TODO: State your research question clearly.

Example: "Can network analysis of ETF correlations predict which ETFs 
will crash most severely during market stress?"


Hypotheses:

TODO: State testable hypotheses.

Example:
H1: ETFs with higher centrality in the correlation network will crash more.
H2: Less liquid ETFs will show stronger contagion effects.


TODO: Implement your analysis


---

## Summary: Research Best Practices

### Good Research Includes:

1. **Clear question**: Well-defined, specific, answerable
2. **Literature review**: What do we already know?
3. **Testable hypotheses**: Predictions that can be falsified
4. **Rigorous methodology**: Justified assumptions, appropriate methods
5. **Sensitivity analysis**: Test how results change with assumptions
6. **Honest limitations**: What doesn't your analysis capture?
7. **Clear presentation**: Visualizations, tables, narrative
8. **Actionable insights**: So what? What should change?

### Common Pitfalls to Avoid:

- **HARKing**: Hypothesizing After Results are Known
- **Cherry-picking**: Only showing favorable results
- **P-hacking**: Trying many tests until something is "significant"
- **Overfitting**: Models that work on your data but not generally
- **Ignoring trade-offs**: Every solution has costs
- **Unrealistic assumptions**: Models must connect to reality

### Publication Venues

If your research yields interesting results:

**Academic Journals**:
- Journal of Finance
- Journal of Financial Economics
- Review of Financial Studies
- Journal of Financial Markets

**Industry Publications**:
- Journal of Trading
- Journal of Portfolio Management
- Risk Magazine

**Working Papers**:
- SSRN (Social Science Research Network)
- arXiv (quantitative finance section)

**Competitions**:
- CFA Institute Research Challenge
- AQR Insight Award

---

## Next Steps

✅ **Completed**: All Extensions Track notebooks

**Continue learning**:
- Read academic papers on market microstructure
- Study other flash crash events (2010, 2020)
- Analyze current market structure developments
- Contribute to this repository with new case studies

**Share your work**:
- Present findings to colleagues or study groups
- Write blog posts explaining your analysis
- Submit pull requests with improvements
- Publish original research

---

*Generated for ETF Flash Crash 2015 Educational Project - Extensions Track*

**Good luck with your research! 🚀**